# **CASE: LIMITE DE CARTÃO DE CRÉDITO**

---

### Instalação de bibliotecas

In [ ]:
# Para geração de correlograma
install.packages("GGally")
# Para cálculo do VIF
install.packages("car")

### Carregamento de bibliotecas

In [ ]:
library(GGally)
library(car)

### Leitura da base de dados

In [ ]:
dados_limite <- read.table(file   = "Limite_Cartao.txt",
                           sep    = "\t",
                           dec    = ",",
                           header = TRUE)

### Visualização da base de dados

In [ ]:
View(dados_limite)

### Análise exploratória breve

In [ ]:
# Medidas resumo
summary(dados_limite)

### Análise bivariada: variável resposta vs. variáveis explicativas

In [ ]:
# Comando auxiliar para aumentar resolução do gráfico (recomendado apenas no Colab)
reset_options <- options()
options(repr.plot.width = 15, repr.plot.height = 15)

In [ ]:
# Correlograma
ggpairs(dados_limite[, !names(dados_limite) %in% "ID_CLIENTE"],
        title = "Correlograma",
        upper = list(continuous = wrap("cor", size = 5)))

In [ ]:
# Resetando parâmetros gráficos (recomendado apenas no Colab)
options(reset_options)

### Modelo de regressão linear múltipla

In [ ]:
# Comando auxiliar para para omitir notação científica nos p-valores e controlar largura dos outputs na tela do Colab
options(scipen = 999, width = 200)

In [ ]:
# Ajuste do modelo 1: inicial
regressao_1 <- lm(LIMITE_INICIAL_CARTAO ~
                  SCORE_CREDITO +
                  VALOR_TOTAL_ATRASOS_12M +
                  QTD_CONSULTAS_CREDITO_12M +
                  RENDIMENTO_MEDIO_12M +
                  TEMPO_TRABALHO +
                  IDADE +
                  QTD_DEPENDENTES,
                data = dados_limite)

summary(regressao_1)

In [ ]:
# Ajuste do modelo 2: retirando QTD_DEPENDENTES
regressao_2 <- lm(LIMITE_INICIAL_CARTAO ~
                  SCORE_CREDITO +
                  VALOR_TOTAL_ATRASOS_12M +
                  QTD_CONSULTAS_CREDITO_12M +
                  RENDIMENTO_MEDIO_12M +
                  TEMPO_TRABALHO +
                  IDADE,
                data = dados_limite)

summary(regressao_2)

In [ ]:
# Ajuste do modelo 3: retirando IDADE
regressao_3 <- lm(LIMITE_INICIAL_CARTAO ~
                  SCORE_CREDITO +
                  VALOR_TOTAL_ATRASOS_12M +
                  QTD_CONSULTAS_CREDITO_12M +
                  RENDIMENTO_MEDIO_12M +
                  TEMPO_TRABALHO,
                data = dados_limite)

summary(regressao_3)

In [ ]:
# Avaliação de colinearidade no modelo 3
data.frame(VIF = vif(regressao_3))

In [ ]:
# Ajuste do modelo 4: retirando VALOR_TOTAL_ATRASOS_12M (colinearidade)
regressao_4 <- lm(LIMITE_INICIAL_CARTAO ~
                  SCORE_CREDITO +
                  QTD_CONSULTAS_CREDITO_12M +
                  RENDIMENTO_MEDIO_12M +
                  TEMPO_TRABALHO,
                data = dados_limite)

summary(regressao_4)

In [ ]:
# Avaliação de colinearidade no modelo 4
data.frame(VIF = vif(regressao_4))

### Intervalos de confiança

In [ ]:
# Escolha do beta (0 para intercepto, ou 1, 2, ... para os ângulos referentes a cada variável explicativa)
beta = 0

In [ ]:
# Cálculo do intervalo
print("Limite inferior")
as.numeric(regressao_4$coefficients[beta + 1] - 1.96 * coef(summary(regressao_4))[beta + 1, "Std. Error"])
print("Limite superior")
as.numeric(regressao_4$coefficients[beta + 1] + 1.96 * coef(summary(regressao_4))[beta + 1, "Std. Error"])

### Qualidade de ajuste

In [ ]:
# Coeficiente de determinação ajustado (R² ajustado)
summary(regressao_4)$adj.r.squared

In [ ]:
# MAE
sum(abs(regressao_4$residuals))/nrow(dados_limite)

In [ ]:
# MAPE
sum(abs(regressao_4$residuals/dados_limite$LIMITE_INICIAL_CARTAO))/nrow(dados_limite)

### Análise de resíduos

In [ ]:
# Histograma dos resíduos
hist(x      = regressao_4$residuals,
     main   = "Histograma dos resíduos",
     col    = "darkturquoise",
     border = "white")

In [ ]:
# QQ plot dos resíduos
qqnorm(y    = regressao_4$residuals,
       main = "QQ-plot dos resíduos",
       col  = "darkturquoise",
       pch  = 19)
qqline(y    = regressao_4$residuals,
       col  = "red")

In [ ]:
# Resíduos vs. valores preditos
plot(x    = regressao_4$fitted.values,
     y    = regressao_4$residuals,
     pch  = 19,
     col  = "darkturquoise",
     xlab = "Valores ajustados",
     ylab = "Resíduos",
     main = "Resíduos vs. valores ajustados")

### Exemplo de uso (aplicação) do modelo

In [ ]:
# Criação de base de dados com um novo cliente, com 80 pontos de score de crédito, 1 consulta de crédito nos últimos 12m, rendimento médio de 5.000 reais e 5 anos de tempo de trabalho
# Obs.: os nomes das colunas devem ser idênticos aos da base utilizada para construção do modelo
novos_dados <- data.frame(SCORE_CREDITO             = c(80),
                          QTD_CONSULTAS_CREDITO_12M = c(1),
                          RENDIMENTO_MEDIO_12M      = c(5000),
                          TEMPO_TRABALHO            = c(5))

In [ ]:
# Aplicação do modelo
novos_dados$LIMITE_INICIAL_CARTAO_PREDITO <- predict(regressao_4, novos_dados)
View(novos_dados)